## 모듈 import

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib import parse # 주소 파싱 위해서
from urllib.parse import quote_plus
import requests
import urllib.request
import Augmentor
import glob, os
from PIL import Image
from sklearn.model_selection import train_test_split
import keras
from keras.utils import to_categorical
import keras.applications.xception as xception
from keras.preprocessing import image
import numpy as np

Using TensorFlow backend.


## 데이터 수집

In [ ]:
getter_url = 'https://thepopularlist.com/most-famous-paintings-in-the-world/'
source = urlopen(url).read()
source = BeautifulSoup(source, 'lxml')

- 데이터를 가져올 주소

In [ ]:
# 첫번째 공백으로 분리하여 추가
keys = []
values = []
fm_name = {}
a = source.find_all('h2')
for i in range(len(a)-2):
    keys.append(a[i+1].text[:a[i+1].text.find(" ")+1])
    values.append(a[i+1].text[a[i+1].text.find(" ")+1:])
    fm_name[keys[i]] =  values[i]

fm_name

- 데이터의 이름들을 확인하기 위한 작업

In [ ]:
c = source.find_all('img',class_='lazy')
str(c[61]).split('\"')[7]
src_url = []
n = 0
for i in range(len(c) - 2):
    n += 1
    print(n, str(c[i+2]).split('\"')[7])
    src_url.append(str(c[i+2]).split('\"')[7])
src_url

In [ ]:
n = 0
for i in range(len(src_url)):
    n+=1
    print(n)
    m = parse.quote(src_url[i][8:])
    try:
        if not os.path.isdir('./fmp'):
            os.mkdir('./fmp')
    except OSError as e :
        if e.errno != errno.EEXIST:
            print('Failed to create Directory!!')
    with urlopen('https://' + m) as f:
        with open('./fmp/' + values[i]+'.jpg','wb') as h: # w - write b - binary
            img = f.read()
            h.write(img)

- 확인한 데이터의 url을 추출하고 위의  url을 통해 이미지를 다운로드

## Data Augmentation

In [ ]:
fmp_name = {}
n = 0
for x in glob.glob("./fmp/*.*g"): 
    n += 1
    fn = os.path.basename(x)
    dn = os.path.basename(x).rsplit('.jpg')[0]
    dn = dn.split('/')[-1]
    fmp_name[n] = dn
    img_src = Image.open(x)
    # resize
    res_img = img_src.resize((299,299)) #max img size (299, 299)
    try:
        if not (os.path.isdir('./resize/')):
            os.mkdir('./resize/')
        if not (os.path.isdir('./resize/'+dn)):
            os.mkdir(os.path.join('./resize/'+dn))
    except OSError as e:
        if e.errno != errno.EExist:
            print("Failed to creat dir")
    res_img.save('./resize/'+dn+'/'+fn, "JPEG", quality = 95)

- 앞서 크롤링을 통해 다운받은 이미지 파일을 리사이징하여 새로 저장해준다.

In [2]:
def augimg(source, output,  sample_no):
    p = Augmentor.Pipeline(source_directory='./'+str(source)+'/', output_directory='./'+'/output/', save_format="JPEG")
    p.flip_left_right(probability=0.2) 
    p.shear(0.3, max_shear_left = 10 , max_shear_right = 10)
    p.random_brightness(0.4, min_factor = 0.2, max_factor = 0.6)
    p.random_color(0.3, min_factor = 0.5, max_factor = 0.8)
    p.random_contrast(0.3, min_factor = 0.5, max_factor = 0.8)
    p.random_distortion(0.5, grid_width = 5, grid_height = 5, magnitude = 10)
    p.black_and_white(0.1) 
    p.rotate(0.3, 5, 20) 
    p.skew(0.3, 0.5) 
    p.zoom(probability = 0.3, min_factor =0.2, max_factor = 0.8)
    p.crop_random(0.3, 0.7)
    p.sample(sample_no) 

In [ ]:
for x in glob.glob('./resize/*/'):
    dn = os.path.dirname(x).split('/')[-1]
    print(dn)
    augimg('./resize/'+dn+'/','output',100)

- 리사이징한 이미지를 토대로 augmentation을 진행 각 그림별로 100개씩 생성

## 이미지 전처리

In [ ]:
X_data = []
y_data = []
n = 0

for x in glob.glob('./resize/*/output/*'):
    dn = os.path.dirname(x).split('/')[-2]
    dn = [key for key, value in fmp_name.items() if value == dn]
    img = image.load_img(x, target_size = (299, 299))
    img_tensor = image.img_to_array(img)
    img_tensor /= 255.
    X_data.append(img_tensor)
    y_data.append(dn[0])
    n += 1
    print(n)

- train data, test data를 만들기 위한 작업

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X_data), y_data, test_size = 0.2)

- 사이킷 런에 포함된 train_test_split 기능을 이용해 무작위로 학습 데이터를 선정한다.

In [ ]:
print('X_train : ', X_train.shape,'X_test : ', X_test.shape, '\ny_train : ', len(y_train), ' y_test : ',len(y_test))

In [ ]:
y_train_cat = to_categorical(np.array(y_train), num_classes = 61)
y_test_cat = to_categorical(np.array(y_test), num_classes = 61)

- 리스트 형태인 y 데이터를 컴퓨터가 학습할 수 있도록 to_categorical을 사용하여 one-hot encoding을 진행

In [ ]:
model = xception.Xception(include_top=True, weights=None, input_tensor=None, input_shape=(299,299,3), pooling='avg', classes=61)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

- xception 모델 로드
    - xception 모델의 경우 imagenet 경연의 우승 모델중 하나로 모델자체가 keras에 구축되어 있다.
    - 이러한 모델을 로드해오는 방법으로 사용
- 모델의 학습을 위해 컴파일 해줌

In [ ]:
X_model = model.fit(X_train, y_train_cat, epochs=200, batch_size=16, validation_data=(X_test, y_test_cat))

- 위의 데이터를 통해 모델을 학습해줌

In [ ]:
model.save('./totalsave.h5')
model.save_weights('./sw_again.h5')

- model.save 는 모델의 구조까지 모두 저장하는 방법으로 기존 로드한 모델에 새로운 구조를 추가시켰을때나 모델의 구조가 변경되었을때 사용할 수 있다.
- model.save_weights 는 모델의 가중치, 즉 학습정도만을 저장하는것으로 모델이 기본 구조를 유지하고 있을때 사용할 수 있다.
    - 전체 저장보다는 가볍고 용량도 작게 차지한다는 장점이 있다.

In [ ]:
model.load_weights('./model0_4.h5')

- 모델의 학습정도만을 불러오는 코드

In [ ]:
a = fmp_name[17]

img = image.load_img('/content/drive/My Drive/Git/second/resize/'+a+'/'+a+'.jpg', target_size=(299,299))

target = []
img_tensor = image.img_to_array(img)
img_tensor /= 255.
target.append(img_tensor)

In [ ]:
x = list(model.predict(np.array(target))[0])

print(max(x))
fmp_name[x.index(max(x))].rsplit(' by')[0]

- 모델의 정확도를 검증하기 위한 확인 코드

In [ ]:
url = 'https://www.google.com/search?q='
paint_name = fmp_name[x.index(max(x))].rsplit(' by')[0]
m = parse.quote(paint_name)
#params = {
#    'q': 'The Embarkation of the Queen of Sheba by Claude Lorrain',
#}

response = requests.get(url+m, headers={'User-Agent': 'Mozilla/5.0'})
html = response.text

soup = BeautifulSoup(html, 'html.parser')
#print(soup)
title_list = soup.find_all('div', class_ = 'BNeawe deIvCb AP7Wnd')
#img_info = soup.find(class_ = 'image').find('img')
#img_src = img_info.get('src')
#title_info = soup.find('table').find_all('td')
#print('http:'+img_src)
#print(title_list)
for tag in title_list:
    print(tag.text)

real_paint_name = title_list[0].text

- 처음 데이터를 가져온 사이트에서 그림의 이름을 축약하여 적어둔 경우가 존재함.
- 이를 처리하기 위해 구글 검색 크롤링을 통해 온전한 이름 획득

In [ ]:
url = 'https://en.wikipedia.org/wiki/'
#m = parse.quote(paint_name)
#params = {
#    'q': 'The Embarkation of the Queen of Sheba by Claude Lorrain',
#}

response = requests.get(url+real_paint_name)
html = response.text

soup = BeautifulSoup(html, 'html.parser')
#print(soup)
title_list_R = soup.find('table').find_all('th')
img_info = soup.find(class_ = 'image').find('img')
img_src = img_info.get('src')
title_info = soup.find('table').find_all('td')
#print('http:'+img_src)
#print(title_list)
for tag,info in zip(title_list, title_info):
    print(tag.text, '  ', info.text)

- 구글 검색을 통해 찾아낸 온전한 이름을 통해 그림의 정보를 크롤링해옴